In [4]:
# ----------------------------------------------------------------------------
#
# TITLE - linear_model_prior.ipynb
# AUTHOR - James Lane
# PROJECT - AST 1501
#
# ----------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Determine the best-fitting linear model with the addition of a prior. Compare to 
data from other sources
'''

__author__ = "James Lane"

In [6]:
# General
import numpy as np
import pdb, sys, importlib, copy, importlib
from tqdm import tqdm_notebook

## Plotting
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation
from matplotlib import colors
from matplotlib import cm

## Astropy
from astropy import units as apu
from astropy.io import fits
from astropy.table import Table

## Scipy
from scipy.stats import binned_statistic_2d, binned_statistic
from scipy.optimize import curve_fit
from scipy import linalg
import scipy as sc

## Galpy
from galpy import potential

## Add project-specific package. Assume relative location
sys.path.append('../../src/')
import ast1501.util
import ast1501.fourier
import ast1501.df
import ast1501.linear_model

In [20]:
### Matplotlib for notebooks
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### Debugging, automatically drop in during error
%pdb
from IPython.core.debugger import set_trace

Automatic pdb calling has been turned ON


# Meta

## Information:

## Future Work:

Test the new ```LinearModel``` class

# Prepare

## Load Data

In [18]:
### Load catalogs
gaiadr2_apogee_catalog = '../../data/generated/gaiadr2-apogee_dataset.FIT'
f = fits.open(gaiadr2_apogee_catalog)
data = f[1].data

### Cut on galactocentric absolute Z < 0.3 kpc
where_low_z = np.where( np.abs(data['Z']) < 0.3 )[0]
data_low_z = data[where_low_z] 
z_select_text = r'$|$Z$_{GC}| < 0.3$ kpc'

### Read catalog values

# ID, RA, Dec, logg, abundances, errors
apid = data_low_z['APOGEE_ID']
locid = data_low_z['LOCATION_ID']
vhelio = data_low_z['VHELIO']
pmll = data_low_z['PM_LL']
pmbb = data_low_z['PM_BB']
gc_x = data_low_z['X']
gc_y = data_low_z['Y']
gc_z = data_low_z['Z']
gc_vx = data_low_z['VX']
gc_vy = data_low_z['VY']
gc_vz = data_low_z['VZ']

%%debug

### Convert to galactocentric radius and radial velocity
gc_R = np.sqrt(np.square(gc_x)+np.square(gc_y))
gc_phi = np.arctan2(gc_y,gc_x)
gc_vR = np.cos(gc_phi)*gc_vx + np.sin(gc_phi)*gc_vy
gc_vT = np.sin(gc_phi)*gc_vx - np.cos(gc_phi)*gc_vy
gc_phi = np.arctan2(gc_y,-gc_x)

--Return--
None
> <ipython-input-18-be36623d4054>(26)<module>()
     24 gc_vz = data_low_z['VZ']
     25 
---> 26 set_trace()
     27 
     28 ### Convert to galactocentric radius and radial velocity

ipdb> gc_vz
array([  0.69913678,  -7.91657268,   8.59446153, ..., -13.48461682,
         7.08558581, -17.71964389])
ipdb> q


BdbQuit: 

## Set Parameters

In [ ]:
# Radial bin range and size
R_lim = [12,15]
R_bin_size = 0.75
R_bin_cents = np.arange( R_lim[0], R_lim[1], R_bin_size ) + R_bin_size/2

# Phi bin range and size
phi_lim = [-np.pi/2, np.pi/2]
phi_bin_size = np.pi/30
phi_bin_cents = np.arange( phi_lim[0], phi_lim[1], phi_bin_size ) + phi_bin_size/2

# Phib bin range and size
phib_lim = [-np.pi/4, np.pi/4]
phib_bin_size = np.pi/60
phib_bin_cents = np.arange( phib_lim[0], phib_lim[1], phib_bin_size ) + phib_bin_size/2


## Try and initialize the class

### Load using Gaia data

In [ ]:
importlib.reload(ast1501.linear_model)

In [ ]:
lm1 = ast1501.linear_model.LinearModel(instantiate_method=1, gc_R=gc_R, 
    gc_phi=gc_phi, gc_vR=gc_vR, gc_vT=gc_vT, R_lims=R_lim, 
    R_bin_size=R_bin_size, phi_lims=phi_lim, 
    phi_bin_size=phi_bin_size, phib_lims=phib_lim, n_bs=100,
    phib_bin_size=phib_bin_size, force_yint_zero_vR=False, use_velocities=['vR','vT'])


In [ ]:
lm1.phiB

In [ ]:
fig, axs = lm1.plot_vRvT_known_m_b_phi(phi_lim=[-np.pi/4,np.pi/2])
plt.show()
plt.close('all')

In [ ]:
plot_kws={'ecolor':'Black','marker':'o','markeredgecolor':'Black','color':'Black','markersize':10,
          'markerfacecolor':'None','capsize':5,'fmt':'o'}
fig, axs = lm1.plot_vRvT_m_r(plot_kws=plot_kws)
plt.show()
plt.close('all')

### Load using DF velocity field filename

In [ ]:
importlib.reload(ast1501.linear_model)

In [ ]:
lm_sfb = ast1501.linear_model.LinearModel(instantiate_method=2, 
    df_filename='../../scripts/generate_bar_df/2019-05-20/data.npy', phib_bin_size=phib_bin_size,
    phib_lims=phib_lim)
lm_lsb = ast1501.linear_model.LinearModel(instantiate_method=2, 
        df_filename='../../scripts/generate_lsbar_df/2019-05-27/data.npy', phib_bin_size=phib_bin_size,
    phib_lims=phib_lim)
lm_th = ast1501.linear_model.LinearModel(instantiate_method=2, 
        df_filename='../../scripts/generate_triaxial_df/2019-03-30/data0.npy', phib_bin_size=phib_bin_size,
    phib_lims=phib_lim)

In [ ]:
fig, axs = lm_sfb.plot_vRvT_known_m_b_phi()

plt.show()
plt.close('all')

In [ ]:
fig, axs = lm_lsb.plot_vRvT_known_m_b_phi()

plt.show()
plt.close('all')

In [ ]:
fig, axs = lm_th.plot_vRvT_known_m_b_phi()

plt.show()
plt.close('all')

In [ ]:
ms=20

# sfb_plot_kws={'ecolor':'Red','marker':'o','markeredgecolor':'Red','color':'Red',
#               'markersize':7,'markerfacecolor':'None','capsize':2,'fmt':'o'}
sfb_plot_kws={'marker':'o','edgecolor':'Red', 's':ms, 'facecolor':'Red'}
fig, axs = lm_sfb.plot_vRvT_m_r(plot_kws=sfb_plot_kws, plot_type='scatter')

# lsb_plot_kws={'ecolor':'DodgerBlue','marker':'o','markeredgecolor':'DodgerBlue','color':'DodgerBlue',
#               'markersize':7, 'markerfacecolor':'None','capsize':2,'fmt':'o'}
lsb_plot_kws={'marker':'o','edgecolor':'DodgerBlue', 's':ms, 'facecolor':'DodgerBlue'}
fig, axs = lm_lsb.plot_vRvT_m_r(plot_kws=lsb_plot_kws, fig=fig, axs=axs, plot_type='scatter')

# th_plot_kws={'ecolor':'Black','marker':'o','markeredgecolor':'Black','color':'Black',
#               'markersize':7, 'markerfacecolor':'None','capsize':2,'fmt':'o'}
th_plot_kws={'marker':'o','edgecolor':'Black', 's':ms, 'facecolor':'None'}
fig, axs = lm_th.plot_vRvT_m_r(plot_kws=th_plot_kws, fig=fig, axs=axs, plot_type='scatter')

axs[1,0].scatter([],[],color='Red',label='SFB')
axs[1,0].scatter([],[],color='DodgerBlue',label='LSB')
axs[1,0].scatter([],[],color='Black',label='TH, b/a=0.9')
axs[1,0].legend()

plt.show()
plt.close('all')